In [1]:
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_b = pd.read_csv("all_bikes.csv")
df_w = pd.read_csv("weather.csv")

/home/wu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_b.head()

,STATION_NUMBER,TIME,STATUS,NUMBER_OF_BIKES,NUMBER_OF_STANDS,NUMBER_OF_SPACES
0,42.0,1.522870e+12,OPEN,24.0,30.0,6.0
1,30.0,1.522870e+12,OPEN,14.0,20.0,6.0
2,54.0,1.522870e+12,OPEN,2.0,33.0,31.0
3,56.0,1.522870e+12,OPEN,15.0,40.0,25.0
4,6.0,1.522870e+12,OPEN,7.0,20.0,12.0


In [4]:
df_w.head()

,temperature,rain,time
0,8.04,0.005,14/03/2018 00:00
1,7.72,0.105,14/03/2018 03:00
2,7.65,0.260,14/03/2018 06:00
3,7.95,0.235,14/03/2018 09:00
4,8.10,0.475,14/03/2018 12:00


In [5]:
df_weather = df_w.drop_duplicates('time')

In [6]:
df_bikes = df_b.drop_duplicates()

Since there is no foreign key in both tables, it's better to add two coulmn with same style of timestamp.

#### For weather table, convent to timestamp

In [7]:
import time
pattern='%d/%m/%Y %H:%M'

In [8]:
#There are a little difference between tamp in weather and bike Since the bike time has accuacy in millseconds
TimestampWeather=[]
for index, row in df_weather.iterrows():
    TimestampWeather.append(int(time.mktime(time.strptime(row['time'],pattern))* 1000))

In [9]:
TimestampWeather[0:5]

[1520985600000, 1520996400000, 1521007200000, 1521018000000, 1521028800000]

In [10]:
df_weather["tamp"] = TimestampWeather

/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_weather.head()

,temperature,rain,time,tamp
0,8.04,0.005,14/03/2018 00:00,1520985600000
1,7.72,0.105,14/03/2018 03:00,1520996400000
2,7.65,0.260,14/03/2018 06:00,1521007200000
3,7.95,0.235,14/03/2018 09:00,1521018000000
4,8.10,0.475,14/03/2018 12:00,1521028800000


#### For bike table,  Add a human readable time in bike table

In [12]:
#drop all the timestamp is NaN
df_bikes = df_bikes[np.isfinite(df_bikes['TIME'])]

In [13]:
time = []
import datetime
for index, row in df_bikes.iterrows():
    time.append(datetime.datetime.fromtimestamp(int(row["TIME"]/1000)).strftime('%d/%m/%Y %H:%M:%S'))

In [14]:
time[0:5]

['04/04/2018 20:29:32',
 '04/04/2018 20:28:54',
 '04/04/2018 20:29:32',
 '04/04/2018 20:27:49',
 '04/04/2018 20:29:27']

In [17]:
df_bikes["time"] = time

In [16]:
import numpy as np
weather_tamp=[]
weather_rain=[]
weather_temperature = []
weather_time =[]
for index, row in df_bikes.iterrows():
    myNumber = row["TIME"]
    closestTime = min(TimestampWeather, key = lambda x : abs(x-myNumber))
    if(abs(myNumber-closestTime) > 60 * 60 * 1000):
        weather_time.append(np.nan)
        weather_rain.append(np.nan)
        weather_temperature.append(np.nan)
        weather_tamp.append(np.nan)
    else:
        weather_tamp.append(closestTime)
        weather_rain.append(df_weather.loc[df_weather['tamp'] == closestTime,'rain'].item())
        weather_temperature.append(df_weather.loc[df_weather['tamp'] == closestTime,'temperature'].item())
        weather_time.append(df_weather.loc[df_weather['tamp'] == closestTime,'time'].item())
    

In [18]:
df_bikes["weather_tamp"] = pd.Series(weather_tamp).values
df_bikes["weather_rain"] = pd.Series(weather_rain).values
df_bikes["weather_temperature"] =pd.Series(weather_temperature).values
df_bikes["weather_time"]=pd.Series(weather_time).values

In [19]:
df_bikes.head(20)

,STATION_NUMBER,TIME,STATUS,NUMBER_OF_BIKES,NUMBER_OF_STANDS,NUMBER_OF_SPACES,time,weather_tamp,weather_rain,weather_temperature,weather_time
0,42.0,1.522870e+12,OPEN,24.0,30.0,6.0,04/04/2018 20:29:32,1.522872e+12,0.0,5.62,04/04/2018 21:00
1,30.0,1.522870e+12,OPEN,14.0,20.0,6.0,04/04/2018 20:28:54,1.522872e+12,0.0,5.62,04/04/2018 21:00
2,54.0,1.522870e+12,OPEN,2.0,33.0,31.0,04/04/2018 20:29:32,1.522872e+12,0.0,5.62,04/04/2018 21:00
3,56.0,1.522870e+12,OPEN,15.0,40.0,25.0,04/04/2018 20:27:49,1.522872e+12,0.0,5.62,04/04/2018 21:00
4,6.0,1.522870e+12,OPEN,7.0,20.0,12.0,04/04/2018 20:29:27,1.522872e+12,0.0,5.62,04/04/2018 21:00
5,18.0,1.522870e+12,OPEN,3.0,30.0,27.0,04/04/2018 20:29:18,1.522872e+12,0.0,5.62,04/04/2018 21:00
6,32.0,1.522870e+12,OPEN,10.0,30.0,20.0,04/04/2018 20:27:21,1.522872e+12,0.0,5.62,04/04/2018 21:00
7,52.0,1.522870e+12,OPEN,1.0,32.0,31.0,04/04/2018 20:25:17,1.522872e+12,0.0,5.62,04/04/2018 21:00
8,48.0,1.522870e+12,OPEN,5.0,40.0,34.0,04/04/2018 20:20:56,1.522872e+12,0.0,5.62,04/04/2018 21:00
9,13.0,1.522870e+12,OPEN,6.0,30.0,24.0,04/04/2018 20:22:36,1.522872e+12,0.0,5.62,04/04/2018 21:00


In [21]:
df_bikes.to_csv("mix.csv")